# Smoothing Test

In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *
from smoothcfg import *

torch.set_default_dtype(torch.float64)

### Test 1: Assigning Constants

In [2]:
text = 'y = gauss(0,1); x = 0;'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]]])>

In [3]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=0,smoothed:x=0+ gm([1.], [0.], [0.0010000000])>,
 'exit': ExitNode<>}

### Test 2: Assigning Discrete Distributions

In [4]:
text = 'y = gauss(0,1); x = bern(0.5);'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist

Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 1.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]]])>

In [5]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0010000000, 0.0010000000])>,
 'exit': ExitNode<>}

### Test 3: Assigning Constant times Discrete

In [6]:
text = 'y = gauss(0,1); x = 2*gm([0.5, 0.5], [0., 1.], [0., 0.]);'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist


Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 2.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 4.0000e-06]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 4.0000e-06]]])>

In [7]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=2*gm([0.5,0.5],[0.,1.],[0.,0.]),smoothed:x=2*gm([0.5,0.5], [0.,1.], [0.0010000000, 0.0010000000])>,
 'exit': ExitNode<>}

### Test 4: Assigning Other Variables

In [8]:
text = 'y = gauss(0,1); x = y;'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist



Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1.0000, 1.0000],
         [1.0000, 1.0000]]])>

In [9]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=y,smoothed:x=y+ gm([1.], [0.], [0.0010000000])>,
 'exit': ExitNode<>}

### Test 5: If Clauses depending on Extreme Discrete Values

In [10]:
text = 'x = bern(0.5); if x == 1 { x = x + 1; } else { x = x - 1; } end if;'

text=compile2SOGA_text(text)
cfg = produce_cfg_text(text)
smooth_cfg(cfg, smooth_eps = 1e-5)
output_dist = start_SOGA(cfg)
output_dist

Dist<['x'],pi: tensor([[5.0000e-01],
        [5.0000e-01],
        [1.4333e-07],
        [1.4333e-07]])
mu: tensor([[ 2.0000e+00],
        [-1.0000e+00],
        [-5.1865e-05],
        [ 5.1865e-05]])
sigma: tensor([[[2.0000e-10]],

        [[2.0000e-10]],

        [[1.0327e-10]],

        [[1.0327e-10]]])>

In [11]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0000100000, 0.0000100000])>,
 'test0': TestNode<test0,x==1,smoothed:x > 1 - 0.0000500000 and x < 1 + 0.0000500000>,
 'state1': StateNode<state1,True,x=x+1,smoothed:x=x+1+ gm([1.], [0.], [0.0000100000])>,
 'state2': StateNode<state2,False,x=x-1,smoothed:x=x-1+ gm([1.], [0.], [0.0000100000])>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

### Test 6: If Clauses depending on Discrete Variables + Double Degenerate update

In [12]:
text = 'x = bern(0.5); if x == 1 { y = 1; } else { y = -1; } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist


Dist<['x', 'y'],pi: tensor([[5.0000e-01],
        [5.0000e-01],
        [1.4333e-07],
        [1.4333e-07]])
mu: tensor([[ 1.0000,  1.0000],
        [ 0.0000, -1.0000],
        [ 0.9948, -1.0000],
        [ 1.0052, -1.0000]])
sigma: tensor([[[9.9999e-07, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[1.0000e-06, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[3.2696e-08, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[3.2696e-08, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]]])>

In [13]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0010000000, 0.0010000000])>,
 'test0': TestNode<test0,x==1,smoothed:x > 1 - 0.0050000000 and x < 1 + 0.0050000000>,
 'state1': StateNode<state1,True,y=1,smoothed:y=1+ gm([1.], [0.], [0.0010000000])>,
 'state2': StateNode<state2,False,y=-1,smoothed:y=-1+ gm([1.], [0.], [0.0010000000])>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [14]:
cfg.smoothed_vars

['x', 'y']

### Test 7: If Clauses depending on intermediate values

In [15]:
text = 'x = gm([0.33, 0.33, 0.34], [0., 1. , 2.], [0., 0., 0.]); if x == 1 { y = 1; } else { y = 0; } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist

Dist<['x', 'y'],pi: tensor([[3.3000e-01],
        [3.3000e-01],
        [9.4595e-08],
        [9.4595e-08],
        [3.4000e-01]])
mu: tensor([[1.0000, 1.0000],
        [0.0000, 0.0000],
        [0.9948, 0.0000],
        [1.0052, 0.0000],
        [2.0000, 0.0000]])
sigma: tensor([[[9.9999e-07, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[1.0000e-06, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[3.2696e-08, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[3.2696e-08, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]],

        [[1.0000e-06, 0.0000e+00],
         [0.0000e+00, 1.0000e-06]]])>

In [16]:
cfg.node_list


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.33,0.33,0.34],[0.,1.,2.],[0.,0.,0.]),smoothed:x=gm([0.33,0.33,0.34], [0.,1.,2.], [0.0010000000, 0.0010000000, 0.0010000000])>,
 'test0': TestNode<test0,x==1,smoothed:x > 1 - 0.0050000000 and x < 1 + 0.0050000000>,
 'state1': StateNode<state1,True,y=1,smoothed:y=1+ gm([1.], [0.], [0.0010000000])>,
 'state2': StateNode<state2,False,y=0,smoothed:y=0+ gm([1.], [0.], [0.0010000000])>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [17]:
cfg.smoothed_vars

['x', 'y']

### Test 8: Observing equalities

In [18]:
text = 'x = gauss(0,1); observe(x==1);'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist

Dist<['x'],pi: tensor([], size=(0, 1))
mu: tensor([], size=(0, 1))
sigma: tensor([], size=(0, 1, 1))>

In [19]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([1.0],[0.000000],[1.000000])>,
 'observe0': ObserveNode<observe0,x==1>,
 'exit': ExitNode<>}

In [20]:
text = 'x = bern(0.5); observe(x==1);'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)
output_dist = start_SOGA(cfg)
output_dist

Dist<['x'],pi: tensor([[1.]])
mu: tensor([[1.]])
sigma: tensor([[[9.9999e-07]]])>

In [21]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000], [0.0,1.0], [0.0010000000, 0.0010000000])>,
 'observe0': ObserveNode<observe0,x==1,smoothed:x > 1 - 0.0050000000 and x < 1 + 0.0050000000>,
 'exit': ExitNode<>}

In [22]:
cfg.smoothed_vars

['x']